# TVL Quality & Sustainability Metrics

This notebook evaluates DeFi protocols based on the *quality* and *durability* of TVL,
not just raw size.

We focus on:
- Growth stability
- Volatility of capital
- Maximum drawdowns
- Capital retention after peak cycles
- TVL half-life

The goal is to distinguish sustainable, usage-driven TVL from incentive-driven liquidity.


In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
RAW_DIR = os.path.join("..", "data", "raw")

files = [f for f in os.listdir(RAW_DIR) if f.startswith("protocol_tvl_daily") and f.endswith(".parquet")]
files.sort()

print("Found files:", len(files))
print("Latest file:", files[-1])


Found files: 1
Latest file: protocol_tvl_daily_20251227_191015.parquet


In [3]:
tvl_path = os.path.join(RAW_DIR, files[-1])
df_tvl = pd.read_parquet(tvl_path)

print("Rows:", len(df_tvl))
print("Columns:", df_tvl.columns.tolist())
df_tvl.head()


Rows: 8497
Columns: ['date', 'protocol', 'slug', 'tvl_usd']


,date,protocol,slug,tvl_usd
0,2020-02-09,Curve DEX,curve-dex,1163733.0
1,2020-02-10,Curve DEX,curve-dex,1372109.0
2,2020-02-11,Curve DEX,curve-dex,8936.0
3,2020-02-12,Curve DEX,curve-dex,38139.0
4,2020-02-13,Curve DEX,curve-dex,273331.0


In [4]:
df_tvl2 = df_tvl.copy()

df_tvl2["date"] = pd.to_datetime(df_tvl2["date"])
df_tvl2 = df_tvl2.sort_values(["protocol", "date"]).reset_index(drop=True)

# Ensure tvl is numeric
df_tvl2["tvl_usd"] = pd.to_numeric(df_tvl2["tvl_usd"], errors="coerce")

df_tvl2.head()


,date,protocol,slug,tvl_usd
0,2020-02-09,Curve DEX,curve-dex,1163733.0
1,2020-02-10,Curve DEX,curve-dex,1372109.0
2,2020-02-11,Curve DEX,curve-dex,8936.0
3,2020-02-12,Curve DEX,curve-dex,38139.0
4,2020-02-13,Curve DEX,curve-dex,273331.0


In [5]:
df_tvl2["ret"] = df_tvl2.groupby("protocol")["tvl_usd"].pct_change()

# Replace inf with NaN (happens if tvl was 0 yesterday)
df_tvl2["ret"] = df_tvl2["ret"].replace([np.inf, -np.inf], np.nan)

df_tvl2[["protocol", "date", "tvl_usd", "ret"]].head(10)


,protocol,date,tvl_usd,ret
0,Curve DEX,2020-02-09,1163733.0,NaN
1,Curve DEX,2020-02-10,1372109.0,0.179058
2,Curve DEX,2020-02-11,8936.0,-0.993487
3,Curve DEX,2020-02-12,38139.0,3.268017
4,Curve DEX,2020-02-13,273331.0,6.166706
5,Curve DEX,2020-02-14,1213755.0,3.440605
6,Curve DEX,2020-02-15,1820510.0,0.499899
7,Curve DEX,2020-02-16,2201773.0,0.209426
8,Curve DEX,2020-02-17,3759265.0,0.707381
9,Curve DEX,2020-02-18,4744228.0,0.262009


## Growth Metrics

Average daily TVL growth is used as a proxy for organic expansion over time.


In [6]:
growth = (
    df_tvl2
    .groupby("protocol")["ret"]
    .mean()
    .rename("avg_daily_growth")
    .reset_index()
)

growth


,protocol,avg_daily_growth
0,Curve DEX,0.010635
1,GMX V2 Perps,0.004101
2,Lido,0.006714
3,Olympus DAO,NaN
4,SushiSwap,0.000061


## TVL Volatility

TVL volatility measures how unstable capital is within a protocol.
Lower volatility suggests stickier, usage-driven liquidity.


In [7]:
volatility = (
    df_tvl2
    .groupby("protocol")["ret"]
    .std()
    .rename("tvl_volatility")
    .reset_index()
)

volatility


,protocol,tvl_volatility
0,Curve DEX,0.181892
1,GMX V2 Perps,0.035531
2,Lido,0.055089
3,Olympus DAO,NaN
4,SushiSwap,0.054697


## Drawdown Analysis

Maximum drawdown captures peak-to-trough TVL losses.



In [8]:
def max_drawdown_from_levels(levels: pd.Series) -> float:
    levels = levels.dropna()
    if len(levels) < 2:
        return np.nan
    peak = levels.cummax()
    dd = (levels - peak) / peak
    return dd.min()

drawdown = (
    df_tvl2
    .groupby("protocol")["tvl_usd"]
    .apply(max_drawdown_from_levels)
    .rename("max_drawdown")
    .reset_index()
)

drawdown


,protocol,max_drawdown
0,Curve DEX,-0.993487
1,GMX V2 Perps,-0.504259
2,Lido,-0.794879
3,Olympus DAO,NaN
4,SushiSwap,-0.987568


In [9]:
coverage = (
    df_tvl2
    .groupby("protocol")["tvl_usd"]
    .apply(lambda s: int(s.notna().sum()))
    .rename("n_days")
    .reset_index()
)

coverage


,protocol,n_days
0,Curve DEX,2143
1,GMX V2 Perps,873
2,Lido,1835
3,Olympus DAO,1740
4,SushiSwap,1906


In [10]:
quality = (
    growth
    .merge(volatility, on="protocol", how="outer")
    .merge(drawdown, on="protocol", how="outer")
    .merge(coverage, on="protocol", how="outer")
)

quality.sort_values("max_drawdown")


,protocol,avg_daily_growth,tvl_volatility,max_drawdown,n_days
0,Curve DEX,0.010635,0.181892,-0.993487,2143
4,SushiSwap,0.000061,0.054697,-0.987568,1906
2,Lido,0.006714,0.055089,-0.794879,1835
1,GMX V2 Perps,0.004101,0.035531,-0.504259,873
3,Olympus DAO,NaN,NaN,NaN,1740


In [11]:
quality_pretty = quality.copy()
quality_pretty["avg_daily_growth"] = quality_pretty["avg_daily_growth"].round(6)
quality_pretty["tvl_volatility"] = quality_pretty["tvl_volatility"].round(6)
quality_pretty["max_drawdown"] = quality_pretty["max_drawdown"].round(6)

quality_pretty.sort_values("max_drawdown")


,protocol,avg_daily_growth,tvl_volatility,max_drawdown,n_days
0,Curve DEX,0.010635,0.181892,-0.993487,2143
4,SushiSwap,0.000061,0.054697,-0.987568,1906
2,Lido,0.006714,0.055089,-0.794879,1835
1,GMX V2 Perps,0.004101,0.035531,-0.504259,873
3,Olympus DAO,NaN,NaN,NaN,1740


In [12]:
# Check Olympus TVL series health
ol = df_tvl2[df_tvl2["protocol"] == "Olympus DAO"].copy()
print("Rows:", len(ol))
print("Non-null TVL days:", ol["tvl_usd"].notna().sum())
print("Non-zero TVL days:", (ol["tvl_usd"].fillna(0) > 0).sum())
print("TVL min/max:", ol["tvl_usd"].min(), ol["tvl_usd"].max())

ol.tail(10)[["date", "tvl_usd"]]


Rows: 1740
Non-null TVL days: 1740
Non-zero TVL days: 0
TVL min/max: 0.0 0.0


,date,tvl_usd
6581,2025-12-19,0.0
6582,2025-12-20,0.0
6583,2025-12-21,0.0
6584,2025-12-22,0.0
6585,2025-12-23,0.0
6586,2025-12-24,0.0
6587,2025-12-25,0.0
6588,2025-12-26,0.0
6589,2025-12-27,0.0
6590,2025-12-27,0.0


## Capital Retention & Half-Life

Retention metrics compare current TVL to historical peaks.

In [13]:
def peak_stats(df):
    df = df.sort_values("date")
    if df["tvl_usd"].dropna().empty:
        return pd.Series({"peak_tvl": np.nan, "peak_date": pd.NaT, "current_tvl": np.nan, "current_vs_peak": np.nan})
    peak_idx = df["tvl_usd"].idxmax()
    peak_tvl = df.loc[peak_idx, "tvl_usd"]
    peak_date = df.loc[peak_idx, "date"]
    current_tvl = df["tvl_usd"].iloc[-1]
    current_vs_peak = current_tvl / peak_tvl if peak_tvl and peak_tvl > 0 else np.nan
    return pd.Series({"peak_tvl": peak_tvl, "peak_date": peak_date, "current_tvl": current_tvl, "current_vs_peak": current_vs_peak})

peak_table = df_tvl2.groupby("protocol").apply(peak_stats).reset_index()
peak_table


C:\Users\David\AppData\Local\Temp\ipykernel_48824\1840273339.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  peak_table = df_tvl2.groupby("protocol").apply(peak_stats).reset_index()


,protocol,peak_tvl,peak_date,current_tvl,current_vs_peak
0,Curve DEX,2.429799e+10,2022-01-05,2.135340e+09,0.087881
1,GMX V2 Perps,6.154133e+08,2025-08-14,3.806784e+08,0.618574
2,Lido,4.252024e+10,2025-08-23,2.569153e+10,0.604219
3,Olympus DAO,0.000000e+00,2021-03-24,0.000000e+00,NaN
4,SushiSwap,7.037692e+09,2021-11-09,1.025940e+08,0.014578


In [14]:
def tvl_half_life_days(df):
    df = df.sort_values("date").dropna(subset=["tvl_usd"])
    if len(df) < 10:
        return np.nan
    
    peak_idx = df["tvl_usd"].idxmax()
    peak_tvl = df.loc[peak_idx, "tvl_usd"]
    peak_date = df.loc[peak_idx, "date"]

    if peak_tvl <= 0:
        return np.nan

    half_level = 0.5 * peak_tvl
    after = df[df["date"] >= peak_date].copy()

    hit = after[after["tvl_usd"] <= half_level]
    if hit.empty:
        return np.nan  # never fell below half
    first_hit_date = hit.iloc[0]["date"]
    return (first_hit_date - peak_date).days

half_life = (
    df_tvl2
    .groupby("protocol")
    .apply(tvl_half_life_days)
    .rename("half_life_days")
    .reset_index()
)

half_life


C:\Users\David\AppData\Local\Temp\ipykernel_48824\937228183.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(tvl_half_life_days)


,protocol,half_life_days
0,Curve DEX,128.0
1,GMX V2 Perps,NaN
2,Lido,NaN
3,Olympus DAO,NaN
4,SushiSwap,61.0


In [ ]:
## Combined Quality Table

In [15]:
quality2 = (
    quality
    .merge(peak_table, on="protocol", how="left")
    .merge(half_life, on="protocol", how="left")
)

# Pretty formatting for screenshot + README
out = quality2.copy()
out["avg_daily_growth"] = out["avg_daily_growth"].round(6)
out["tvl_volatility"] = out["tvl_volatility"].round(6)
out["max_drawdown"] = out["max_drawdown"].round(6)
out["current_vs_peak"] = out["current_vs_peak"].round(4)

out.sort_values("max_drawdown")


,protocol,avg_daily_growth,tvl_volatility,max_drawdown,n_days,peak_tvl,peak_date,current_tvl,current_vs_peak,half_life_days
0,Curve DEX,0.010635,0.181892,-0.993487,2143,2.429799e+10,2022-01-05,2.135340e+09,0.0879,128.0
4,SushiSwap,0.000061,0.054697,-0.987568,1906,7.037692e+09,2021-11-09,1.025940e+08,0.0146,61.0
2,Lido,0.006714,0.055089,-0.794879,1835,4.252024e+10,2025-08-23,2.569153e+10,0.6042,NaN
1,GMX V2 Perps,0.004101,0.035531,-0.504259,873,6.154133e+08,2025-08-14,3.806784e+08,0.6186,NaN
3,Olympus DAO,NaN,NaN,NaN,1740,0.000000e+00,2021-03-24,0.000000e+00,NaN,NaN


## Key Takeaways

- Curve and SushiSwap exhibit extreme drawdowns, consistent with incentive-heavy liquidity cycles.
- GMX V2 shows lower volatility and shallower drawdowns, indicating more durable TVL.
- Lido maintains strong capital retention despite market cycles, suggesting utility-driven deposits.
- Olympus DAO illustrates cases where TVL is no longer a meaningful health metric.

This analysis shows why raw TVL rankings alone are insufficient for protocol evaluation.
